# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

This is a classification problem. We are looking for a true/false discrete answer not a continuous measurement of varying degree, which would fit better with a regression.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [3]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [4]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns) - 1

# TODO: Calculate passing students
n_passed = len(student_data[student_data.passed == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data.passed == 'no'])

# TODO: Calculate graduation rate
grad_rate = 100*float(n_passed)/float(n_students)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [5]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [6]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
#line param.bharat@gmail.com suggested I use
#y_all = y_all.replace(['yes','no'],[1,0])        # <-- I added this line
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [7]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
    
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train = None
X_test = None
y_train = None
y_test = None

X_train, X_test, y_train, y_test = train_test_split(X_all, 
                                                    y_all, 
                                                    test_size = num_test, 
                                                    train_size = num_train,
                                                    random_state = 42
                                                   )

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

I'm currently thinking the following are good learning models:

* Decision Tree
* K-Nearest Neighbors
* AdaBoost

### Decision Tree
#### Real World Application
Decision trees have been used to predict which books in a library can be stored in off-site low-cost storage and which books should remain in the library.

_Reference:_
Silverstein, C. and Shieber, S. M. Predicting individual book use for off-site storage using decision
trees. _Lib_.1Q. 66(3):266–293,-1996.

#### Strengths of the Model

Decision trees are easier to and understand and visualize. Decisions trees also perform well even if its assumptions are sometimes violated by the true underlying model that generated the data.

For classification, decision trees are very fast at predicting.

#### Weaknesses of the Model
Decision trees can be unstable due to small variations in the data that can result in completely different trees being generated.

Learning the optimal tree is NP-complete. Thus most decision tree learners use heuristics to generate trees and usually generate locally optimal trees.

#### Why is this a good candidate for this problem
The amount of data is small. I also hypothesize that there are very few attributes that yield most of the information gain so I'm thinking the time to create the learner will be small.


### K-NN
#### Real World Application
KNN has been used in CRM to predict how much money a new customer will spend on mobile data services.

_Reference_:
RADA, Rexhep; RUSETI, Bashkim. Some Applications of K-Nearest Neighbor Algorithm In Customer Relationship Management.
#### Strengths of the Model
KNN is very effective at training noisy data and is good at training very large data sets.

#### Weaknesses of the Model
Inference of KNN algorithms is slower than other learning algorithms (lazy algorithm). The accuracy of a KNN classifier is dependent on the quality of the distance function. The distance function quality is a function of domain knowledge. 

#### Why is this a good candidate for this problem
I think that students that pass or fail have a lot in common with each other (they are close to each other) and the number of features they have in common is small relative to the total number of features. I also don't think it's linear so I think an instance-based approach is a good choice.

### AdaBoost
#### Real World Application
AdaBoost has been used to score credit worthiness of potential borrowers.

_Reference_:
Edwards,Paul K.; Dhuhon, Dina; Shergill,  Suhail. 
Real AdaBoost: boosting for credit scorecards and similarity to WOE logistic regression. SAS Global Forum 2017. Paper 1323-2017.

#### Strengths of the Model
Boosting learns fast. Is able to make use of other learning algorithms and combine weak classifiers of these learning algorithms and create arbitrarly accurate classifiers. Another interesting strength of boosting is that it can exhibit increasing ability to generalize even after it is no longer able to increase accuracy over a training data set.
#### Weaknesses of the Model
If the underlying weak classifier overfits, then the boosted form of the algorithm will over fit and cannot recover from this.
#### Why is this a good candidate for this problem
I can combine simple decision trees together via boosting to create a very accurate classifier. I'm concerned that I may not be able to identify which features to put into a single decision tree to result in a single highly accurate classifier but with boosting, I can use the algorithm to create a set of simple decision trees that automatically pick the right features.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [8]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')
    #return f1_score(target.values, y_pred, pos_label=1)


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [12]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn import tree

# from sklearn import model_B
from sklearn import neighbors

#KNN classifier initialization parameters
knnNeighbors = 5
knnWeights = "uniform"

# from sklearn import model_C
from sklearn.ensemble import AdaBoostClassifier

#AdaBoost initialization parameters
abNoOfEstimators = 50

# TODO: Initialize the three models
clf_A = tree.DecisionTreeClassifier(random_state=42)
clf_B = neighbors.KNeighborsClassifier(knnNeighbors, knnWeights)
clf_C = AdaBoostClassifier(n_estimators=abNoOfEstimators,
                            algorithm='SAMME', 
                            random_state=42)



for myclf in [clf_A, clf_B, clf_C]:
    for mysize in [100, 200, 300]:

        #for classifier A
        train_predict(myclf, X_train[:mysize], y_train[:mysize], X_test, y_test)
        print "\n"
        
    print "\n\n"





Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0014 seconds
Made predictions in 0.0008 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.6552.


Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0020 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7500.


Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0023 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.6613.





Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0005 seconds
Made predictions in 0.0016 seconds.
F1 score for training set: 0.8060.
Made predictions in 0.0013 seconds.
F1 score for test set: 0.7246.


Training a KNeighborsCl

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - DecisionTreeClassifier**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0014 seconds  | 0.0008 seconds         | 1.0              |     0.6552       |
| 200               |         0.0020  seconds | 0.0006 seconds         | 1.0              |     0.75      |
| 300               |         0.0023 seconds  | 0.0003 seconds         | 1.0              |    0.6613      |

** Classifer 2 - KNeighborsClassifier**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0005 seconds      |  0.0016 seconds        |  0.8060            |    0.7246       |
| 200               |     0.0006 seconds      |  0.0032 seconds        |  0.880            |     0.7692      |
| 300               |     0.0010 seconds      |  0.0079 seconds        |  0.8809             |    0.7801         |

** Classifer 3 - AdaBoostClassifier**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0965 seconds        | 0.0026 seconds         |  0.8794          | 0.7714          |
| 200               |   0.0978 seconds        | 0.0042 seconds         |  0.8814          | 0.7647          |
| 300               |   0.1007 seconds        | 0.0035 seconds         |  0.8501          | 0.7647          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

I suggest that we use the AdaBoostClassifier trained on 200 of the 395 students. My reason for selecting this classifier is because it has the highest score of any of the classifiers trained over any of the data sets we created. Although relatively speaking, this classifier is much slower at training and even predicting than the other classifiers it's still an acceptable performance given the size of the dataset. Even if the dataset were much larger, I think it would continue to be acceptable.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

The AdaBoost model is an "ensemble prediction model." By that I mean that it calculates a prediction by combining the predictions of several simpler models that it automatically creates. Each sub-model prediction is weighted according to how hard it had to work to come up with that prediction.

Here's an example to describe AdaBoost's learning process:

Let's say we begin with a set of ten data items as shown below

![step0](step0.jpg)

The green +'s and red -'s are the real labels of these items. AdaBoost learns by generating in a step by step process classifiers that are just a little bit better than random guessing i.e., better than 50% by some margin. That margin we will call the error rate.

So in the first step, Adaboost generates a classifier h1 that works like this:

![step1](step1.jpg)

All items to the left are predicted by h1 as +'s, all items to the right are predicted as -'s. As you can see h1 gets 3 of the 10 items incorrect. At step one it thinks all items are equally important, so the error rate at step one is 0.3. 

Now the strength of Adaboost is that it will use this error to calculate weights for the items that it classifies incorrectly. These weights will be used to judge how well the subsequent classifier must work to be accepted for that next step. I will try to express this in the drawing by drawing items that have an increased weight larger than items that have an unchanged weight, and items whose weight has increased as smaller. Also I will draw items smaller or bigger from one step to the next to try to show how the weights change.

Here is the result after AdaBoost generates a learner for step 2, called h2:

![step2](step2.jpg)

In step 2, the learner gets 5 items correct and 3 items wrong. You may think that it has again an error rate of 0.3. In fact the error rate is 0.21 because it was able to in this step guess the 3 items it got wrong correctly in step 1 that are now very important to get right in this current step. The three items that are important are the three green pluses in the upper right of the square that I drew a little larger to express their increased importance to the algorithm.

AdaBoost then proceeds to step 3 as shown below:

![step3](step3.jpg)

In step 3, AdaBoost generates classifier h3. This time it only guesses two items incorrectly (one minus and one plus). It's error rate is better than 0.2 because it guessed correctly the three -'s it got wrong in step 2 (the three larger minues in the bottom center of the square) which consequently have become more important to get correct in this step, while at the same time continuing to correctly guess items it got wrong in the past in this current step.

These three steps continue until it's not possible to generate a classifier that provides a better error rate, or when you've hit the required acccuracy, or you've run out of time.

When the learning process described above is complete, AdaBoost then combines the classifiers h1, h2, h3, etc. into a single classifier H. This classifier is simply a *weighted sum* of the little h's. The weights are those that were produced as part of the learning process described above.



### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [15]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from   sklearn.grid_search import GridSearchCV
from sklearn.metrics import explained_variance_score, make_scorer
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier


# TODO: Create the parameters list you wish to tune
parameters = {'base_estimator' : [
            DecisionTreeClassifier(max_depth=2),
            DecisionTreeClassifier(max_depth=5), DecisionTreeClassifier(max_depth=10)],
             'n_estimators' : [150,250,350]}

# TODO: Initialize the classifier
clf = AdaBoostClassifier(n_estimators=50,algorithm='SAMME')

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,parameters,f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0126 seconds.
Tuned model has a training F1 score of 1.0000.
Made predictions in 0.0081 seconds.
Tuned model has a testing F1 score of 0.7867.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

The final F1 training score is 1.000 and the final F1 testing score is 0.7867. For the untuned model the F1 training score was 0.85 and F1 testing score was 0.7647. The results of the tuned model are better than the those of the untuned model.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.